In [22]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def stats_metric(top, bottom, MAX_DIST=50):

    # big, small = None , None
    # if s1[3] > s2[3]:
    #     big = s1
    #     small = s2
    # else:
    #     big = s2
    #     small = s1
    # top, bottom = None, None
    # if s1[1] < s2[1]:
    #     top = s1
    #     bottom = s2
    # else:
    #     top = s2
    #     bottom = s1
    diff = abs(top[1] + top[3] - bottom[1])
    
    return diff < MAX_DIST

cap = cv2.VideoCapture ('vid1_IR.avi')
while (cap.isOpened()):
    ret, frame = cap.read()
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, bin = cv2.threshold(G, 45, 255, cv2.THRESH_BINARY)
    bin = cv2.GaussianBlur(bin, (5, 5), 0)
    bin = cv2.morphologyEx(bin, cv2.MORPH_CLOSE, np.ones((5, 5)))

    numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        bin)
    
    MAX_DIST=25
    for s1 in range(1, stats.shape[0] - 1):
        for s2 in range(s1+1, stats.shape[0]):
            top, bottom = None, None
            if stats[s1, 1] < stats[s2, 1]:
                top = stats[s1, :]
                bottom = stats[s2, :]
            else:
                top = stats[s2, :]
                bottom = stats[s1, :]

            diff_y = abs(top[1] + top[3] - bottom[1])
            diff_x = abs(top[0] - bottom[0])
            if diff_y < MAX_DIST and diff_x < 2*diff_x:
                top[3] += diff_y + bottom[3]
                bottom[3] = 0


    
    I_VIS = bin
    if (stats.shape[0] > 1): # are there any objects
        for pi in range(1, stats.shape[0]):
            if stats[pi, 4] > 2000 and stats[pi, 3] / stats[pi, 2] > 1.5:
                cv2.rectangle(I_VIS, (stats[pi, 0], stats[pi, 1]), (stats[pi, 0] + stats[pi, 2], 
                                    stats[pi , 1] + stats[pi, 3]), (255, 0, 0), 2)
    # cv2.imshow('IR', bin)
    cv2.imshow('IR', I_VIS)
    if cv2.waitKey(1) & 0xFF == ord('q'): # break the loop when the ’q’ key is pressed
        break
cap.release ()